In [2]:
var ds = require('dstools');
var input = require('../examples/data/3features/input-m.json'); 
var diversibute = require('../index');

#### Wie schneiden die beiden Algorithmen bei 100x Tests ab?  

Gruppengröße: 26  
Gruppenanzahl: 3

In [17]:
var groups = 5;

var tests = 50;

In [11]:
// 50x runs of monte carlo
var scoresMonteCarlo = [];
var resultsMonteCarlo = {};

// Initialize empty promise collection
var runs = [];

for(let run = 0; run < tests; run++) {
    
    // Calculate groups with monte carlo
    runs.push(diversibute.monteCarlo(input, groups).then(result => {
                
        // Collect result and score
        resultsMonteCarlo[result.score] = result;
        scoresMonteCarlo.push(result.score);
        
        return;
    }));
}

var highscoreMonteCarlo = undefined;

// Run async tasks in a sequence
runs.reduce((promiseChain, currentTask) => {    
    return promiseChain.then(chainResults =>        
             currentTask.then(currentResult =>            
                  [ ...chainResults, currentResult ]        
             )    
        );
    },Promise.resolve([])).then(arrayOfResults => {    

        // Do something with all results
        console.log(scoresMonteCarlo);

        // Calculate highest score of this batch
        highscoreMonteCarlo = Math.max(...scoresMonteCarlo);

        console.log(highscoreMonteCarlo);
    });

[
  21.973638186555604,   26.1282288514046,  45.24326647573308,
   37.59287231531427,  30.66717964323027, 36.082420549722684,
   42.34404496023048, 26.855037532280036,  44.27576585485039,
   32.06081368915922, 47.074441495648955, 22.197270250281946,
   50.47745110383772, 43.128050337251224,  34.89423807392524,
   35.71122253179824, 46.417508162425776,  36.97380077011019,
  30.435639389603338,  44.87473951765567,  43.51626037123455,
   21.18071380850232,  50.06500407651998, 47.031779775563365,
   59.81901522829877,  37.67567151284936, 12.425120561704887,
   41.96807963191284, 19.622204365574248,  37.68112099767312,
   33.37862820017192,  23.96488986281304, 28.022036707030864,
   36.45592709368607,  47.43087596331114, 53.833600823037486,
   26.94505746266448,  32.71864140529544, 21.240370193266827,
  40.216571642840606, 55.627472860294375, 25.806810377361415,
   53.12856231131279,  37.25525124788406,  37.72642785390505,
   51.99142453909873, 30.662270582115568,  32.32393055537494,
   39.

In [18]:
// 50x runs of genetic
var scoresGenetic = [];
var resultsGenetic = {};
var runs = [];

var settingsGenetic = {populationStartSize: (Object.keys(input).length * 2), 
                       populationMaxSize: (Object.keys(input).length * 3), 
                       evolutions: 1000};

// Run algorithm x times
for(let run = 0; run < tests; run++) {
    runs.push(diversibute.genetic(input, groups, settingsGenetic).then(result => {
        
        // Collect result and score
        resultsGenetic[result.score] = result;
        return scoresGenetic.push(result.score);
    }));
}

var highscoreGenetic = undefined;

// We must wait until all calculations are done
Promise.all(runs).then(values => {
    
    console.log(scoresGenetic);

    // Calculate highest score of this batch
    highscoreGenetic = Math.max(...scoresGenetic);

    console.log(highscoreGenetic);
});

[
  76.03633442867398, 72.21764200146241, 77.42219638582125,
  79.77934933935803, 74.40877044180615, 75.63553322511322,
  79.65113719150071, 75.59378407718438, 78.02520574355827,
  78.88015268166293,  71.2656904293639, 76.57618425981819,
  77.47780899554657, 74.70982868583367, 74.48078850959267,
  80.56542348559819, 81.11276643773165, 75.92177391408381,
  71.59546330708542, 80.60694065025496, 76.56367088673053,
  79.57506573891317, 76.63535138322796, 77.34898642368425,
  77.52307585021556, 80.27351403581085, 71.38001598817607,
  75.66618935221962, 78.29226310060399, 77.69226887694226,
  77.28701766426244, 76.66810066397544, 76.78956004272675,
  78.41488472382756, 77.59297737131449, 81.59074658128068,
  79.13954850063081,   79.132583245577, 77.24075145212866,
  76.37610995005194, 74.71388701291468,  77.8479857295296,
  79.41279200484308,  77.5560934068243, 77.34104631294024,
  72.16533877373027, 79.07198961333305, 78.15073416125915,
  79.92617064540798, 76.36162627988793
]
81.5907465812

## Vergleich Algorithmen

In [19]:
// Calculate overall hitrate

// Highest score
var highestScore = (highscoreMonteCarlo > highscoreGenetic) ? highscoreMonteCarlo : highscoreGenetic;

var hitRateMonteCarlo = scoresMonteCarlo.map(v => v/highestScore);
var hitRateGenetic = scoresGenetic.map(v => v/highestScore);

In [20]:
var trace1 = {
  x: hitRateMonteCarlo,
  type: "histogram",
  opacity: 0.5,
  marker: {
     color: 'green',
  },
  name: 'Monte Carlo'
};
var trace2 = {
  x: hitRateGenetic,
  type: "histogram",
  opacity: 0.6,
  marker: {
     color: 'red',
  },
  name: 'Genetic Algorithm'
};

var data = [trace1, trace2];
var layout = {barmode: "overlay", xaxis: {title: 'Trefferquote'}, yaxis: {title: 'Häufigkeit'}};

ds.Collection().plotly(data, layout).show();

## Darstellung "bestes" Ergebnis

In [15]:
// Retrieve winner combination
var winnerSet = (highestScore == highscoreMonteCarlo) ? resultsMonteCarlo : resultsGenetic;
var winner = winnerSet[highestScore];

// Generate empty traces
/*var traces = ['rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)'].map(c => {
    return {x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
            color: c,
            width: 0.5},
            opacity: 0.8},
        type: 'scatter3d'};
});
*/
var traces = [];
for(let i = 0; i < groups; i++) {
    traces.push({x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
                color: 'rgba(217, 217, 217, 0.14)',
                width: 0.5
            },
            opacity: 0.8
        },
        type: 'scatter3d'
    });
}

// Loop input data and build group traces
Object.keys(input).forEach((member, index) => {
    
    // Retrieve member group 
    var group = winner.combination[index];
    
    traces[group-1].x.push(input[member][0]);
    traces[group-1].y.push(input[member][1]);
    traces[group-1].z.push(input[member][2]);    
});

// Print 3D Model with group traces and colors
ds.Collection().plotly(traces,{margin: {l: 0, r: 0, b: 0, t: 0 }}).show();